In [1]:
pwd

u'/data04/notebooks/vish.ai/phonepe'

In [2]:
#Importing Libraries
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

#from sklearn.externals import joblib

In [3]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SQLContext, HiveContext
from pyspark.storagelevel import StorageLevel
from ConfigParser import ConfigParser
import sys
import datetime
import commands
from dateutil import parser
from dateutil.relativedelta import relativedelta
from pyspark.sql import functions as F
from pyspark.sql.functions import substring
from pyspark.sql import types as T,Window
from pyspark import Row
from datetime import datetime, timedelta
import sys

In [5]:
df = pd.read_excel("PhonePe_Sherloc_Categories.xlsx")  # Replace with the path to your dataset
df.columns

Index([u'upi_transaction_id', u'transaction_type', u'transaction_reference_id',
       u'customer_reference_number', u'note', u'payer_account_number',
       u'payer_account_type', u'payer_ifsc', u'payer_name', u'payer_vpa',
       u'payer_amount', u'payee_account_number', u'payee_account_type',
       u'payee_ifsc', u'payee_name', u'payee_vpa', u'payee_amount', u'state',
       u'created', u'updated', u'data_dt', u'phone_number', u'account_number',
       u'account_type', u'account_ifsc', u'account_holder_name',
       u'mbanking_enabled', u'registered_at_acct', u'Category1', u'Category2'],
      dtype='object')

In [6]:
df2 = df[['payer_name','payer_vpa','payee_account_type','payee_name','payee_vpa','payer_account_type', 'Category1','Category2']]

In [7]:
def custom_tokenizer(text):
    # split the text and value using regular expression
    import re
    pattern = re.compile(r'[a-zA-Z]+\d+')
    text_and_value = pattern.findall(text)
    return text_and_value
# Apply TF-Vectorization on data 
tfidf_payer_name = TfidfVectorizer()
tfidf_matrix_payer_name = tfidf_payer_name.fit_transform(df2['payer_name'].astype(str))

tfidf_payee_name = TfidfVectorizer()
tfidf_matrix_payee_name = tfidf_payee_name.fit_transform(df2['payee_name'].astype(str))

tfidf_payee_account_type = TfidfVectorizer()
tfidf_matrix_payee_account_type = tfidf_payee_account_type.fit_transform(df2['payee_account_type'].astype(str))

tfidf_payer_account_type = TfidfVectorizer()
tfidf_matrix_payer_account_type = tfidf_payer_account_type.fit_transform(df2['payer_account_type'].astype(str))

tfidf_payer_vpa = TfidfVectorizer(tokenizer=custom_tokenizer)
df2['payer_vpa'] = df2['payer_vpa'].astype(str)
tfidf_matrix_payer_vpa = tfidf_payer_vpa.fit_transform(df2['payer_vpa'])

tfidf_payee_vpa = TfidfVectorizer(tokenizer=custom_tokenizer)
df2['payee_vpa'] = df2['payee_vpa'].astype(str)
tfidf_matrix_payee_vpa = tfidf_payee_vpa.fit_transform(df2['payee_vpa'])


tfidf_matrix = pd.concat([pd.DataFrame(tfidf_matrix_payer_name.toarray()),
                          pd.DataFrame(tfidf_matrix_payee_name.toarray()), 
                          pd.DataFrame(tfidf_matrix_payee_account_type.toarray()), 
                          pd.DataFrame(tfidf_matrix_payer_account_type.toarray()),
                          pd.DataFrame(tfidf_matrix_payer_vpa.toarray()),
                          pd.DataFrame(tfidf_matrix_payee_vpa.toarray())], axis=1)

/opt/anaconda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/anaconda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
X_train, X_test, y_cat1_train, y_cat1_test, y_cat2_train, y_cat2_test = train_test_split(tfidf_matrix, df2['Category1'], df2['Category2'], test_size=0.2, random_state=42)

In [9]:
clf_cat1 = RandomForestClassifier() 

clf_cat1.fit(X_train, y_cat1_train) 

clf_cat2 = RandomForestClassifier() 
clf_cat2.fit(X_train, y_cat2_train) # Make predictions for each target variable 

predictions_cat1 = clf_cat1.predict(X_test) 
predictions_cat2 = clf_cat2.predict(X_test)

In [18]:
#class 1 accuracy
accuracy = clf_cat1.score(X_test, y_cat1_test)
print('Accuracy:', accuracy)

('Accuracy:', 0.97218863361547758)


In [19]:
#class 2 accuracy
accuracy = clf_cat2.score(X_test, y_cat2_test)
print('Accuracy:', accuracy)

('Accuracy:', 0.96070133010882708)


In [15]:
df2.head(2)

,payer_name,payer_vpa,payee_account_type,payee_name,payee_vpa,payer_account_type,Category1,Category2
0,H VIGNESH,7019270274@ybl,SAVINGS,PAVAN BAR,gpay-11200233635@okbizaxis,SAVINGS,Personal Transfer,TRANSFER_FRIENDS_AND_FAMILY
1,H VIGNESH,7019270274@ybl,SAVINGS,SANJEEVA C,Q513055970@ybl,SAVINGS,Personal Transfer,TRANSFER_FRIENDS_AND_FAMILY


In [11]:
import pickle

# Save the trained TF-IDF vectorizers to pickle files
with open('pickle_letest/payer_name.pkl', 'wb') as file:
    pickle.dump(tfidf_payer_name, file)
    
with open('pickle_letest/payer_vpa.pkl', 'wb') as file:
    pickle.dump(tfidf_payer_vpa, file)

with open('pickle_letest/payee_account_type.pkl', 'wb') as file:
    pickle.dump(tfidf_payee_account_type, file)

with open('pickle_letest/payee_name.pkl', 'wb') as file:
    pickle.dump(tfidf_payee_name, file)

with open('pickle_letest/payee_vpa.pkl', 'wb') as file:
    pickle.dump(tfidf_payee_vpa, file)

with open('pickle_letest/payer_account_type.pkl', 'wb') as file:
    pickle.dump(tfidf_payer_account_type, file)
    
with open('pickle_letest/classifier1.pkl', 'wb') as file:
    pickle.dump(clf_cat1, file)
    
with open('pickle_letest/classifier2.pkl', 'wb') as file:
    pickle.dump(clf_cat2, file)

In [21]:
payer_name="H VIGNESH"
payer_vpa = "7019270274@ybl"
payee_account_type="SAVINGS"
payee_name="SANJEEVA C"
payee_vpa="Q513055970@ybl"
payer_account_type="SAVINGS"


In [22]:
def get_predictions(payer_name, payer_vpa, payee_account_type, payee_name, payee_vpa, payer_account_type):
    payer_name_tfidf = tfidf_payer_name.transform([payer_name])
    payer_vpa_tfidf = tfidf_payer_vpa.transform([payer_vpa])
    payee_account_type_tfidf = tfidf_payee_account_type.transform([payee_account_type])
    payee_name_tfidf = tfidf_payee_name.transform([payee_name])
    payee_vpa_tfidf = tfidf_payee_vpa.transform([payee_vpa])
    payer_account_type_tfidf = tfidf_payer_account_type.transform([payer_account_type])

    input_tfidf = pd.concat([pd.DataFrame(payer_name_tfidf.toarray()),
                             pd.DataFrame(payer_vpa_tfidf.toarray()),
                             pd.DataFrame(payee_account_type_tfidf.toarray()),
                             pd.DataFrame(payee_name_tfidf.toarray()),
                             pd.DataFrame(payee_vpa_tfidf.toarray()),
                             pd.DataFrame(payer_account_type_tfidf.toarray())], axis=1)
    
    predictions1 = clf_cat1.predict(input_tfidf)
    predictions2 = clf_cat2.predict(input_tfidf)
    
    predictions = pd.DataFrame({'category_level1':predictions1,'category_level2':predictions2})
    
    return predictions

predictions = get_predictions(payer_name, payer_vpa, payee_account_type, payee_name, payee_vpa, payer_account_type)
print(predictions)

  category_level1 category_level2
0        Shopping        Shopping
